In [1]:
!pip install -U bitsandbytes


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import transformers
import inseq
import torch
import pandas as pd

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
transcripts = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_transcripts.csv')

In [4]:
transcripts.head()

,comedian,transcript
0,Anthony_Jeselnik,"When I was a kid, I used to fantasize about ge..."
1,Anthony_Jeselnik_2,No one should ever ask me to speak at anyone’...
2,Ali_Wong,"The last time I was at home in San Francisco, ..."
3,Ali_Wong_2,I need to have children to keep me company whe...
4,Chelsea_Peretti,A friend of a friend just posted like 500 enga...


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ada/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import nltk.data

def split_transcript(transcript):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = tokenizer.tokenize(transcript.strip())
    total_sentences = len(sentences)
    sentences_per_part = total_sentences // 4

    part1 = sentences[:sentences_per_part]
    part2 = sentences[sentences_per_part:2*sentences_per_part]
    part3 = sentences[2*sentences_per_part:3*sentences_per_part]
    part4 = sentences[3*sentences_per_part:]

    part1_text = ' '.join(part1)
    part2_text = ' '.join(part2)
    part3_text = ' '.join(part3)
    part4_text = ' '.join(part4)

    return [part1_text, part2_text, part3_text, part4_text]


In [7]:
transcript_ali_wong = transcripts.loc[transcripts['comedian'] == 'Ali_Wong', 'transcript'].values[0]
parts = split_transcript(transcript_ali_wong)

In [8]:
Q8BIT = transformers.BitsAndBytesConfig(load_in_8bit=True)

model = inseq.load_model(
    model="google/gemma-2b-it", 
    attribution_method="input_x_gradient",
    model_kwargs=dict(
        device_map="cuda:0",
        torch_dtype=torch.float32,
        trust_remote_code=False,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
        quantization_config=Q8BIT,
    )
)

print("memory cost: ", model.model.get_memory_footprint() >> 20, "Mb")

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.98s/it]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.


memory cost:  3890 Mb


In [9]:
prompt = f"This is a stand-up comedy transcript: '''{parts[1]}'''\nExtract the key humorous lines and punchlines:"

out = model.attribute(
prompt,
n_steps=16,
batch_size=1,
generation_args=dict(
    max_new_tokens=65,
)
)
()
out.show()

/home/ada/humor/.venv/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 142/142 [00:34<00:00,  1.65it/s]


,,Sure,",",▁here,▁are,▁the,▁key,▁humorous,▁lines,▁and,▁punch,lines,▁from,▁the,▁stand,-,up,▁comedy,▁transcript,:,,-,▁She,▁refused,▁to,▁let,▁go,▁of,▁a,▁Texas,▁Instruments,▁TI,-,8,2,…,▁manual,.,,-,▁Those,▁of,▁you,▁under,▁,2,5,▁probably,▁don,’,t,▁know,▁what,▁that,▁calculator,▁is,.
<bos>,0.2,0.067,0.077,0.103,0.063,0.072,0.07,0.068,0.114,0.093,0.052,0.074,0.082,0.088,0.05,0.078,0.085,0.067,0.081,0.071,0.13,0.095,0.054,0.068,0.064,0.083,0.055,0.057,0.088,0.071,0.076,0.046,0.068,0.062,0.076,0.038,0.07,0.076,0.039,0.04,0.041,0.062,0.059,0.062,0.071,0.086,0.055,0.048,0.052,0.038,0.057,0.08,0.062,0.101,0.042,0.072,0.048
This,0.113,0.028,0.029,0.038,0.023,0.03,0.013,0.012,0.019,0.009,0.035,0.036,0.026,0.028,0.023,0.024,0.061,0.017,0.033,0.021,0.072,0.021,0.009,0.006,0.018,0.006,0.028,0.022,0.006,0.007,0.027,0.023,0.046,0.04,0.021,0.005,0.008,0.036,0.014,0.03,0.01,0.012,0.018,0.029,0.052,0.006,0.032,0.025,0.034,0.01,0.039,0.006,0.021,0.008,0.019,0.026,0.031
▁is,0.006,0.006,0.006,0.006,0.007,0.006,0.005,0.005,0.004,0.004,0.003,0.002,0.008,0.012,0.012,0.012,0.008,0.009,0.015,0.008,0.005,0.004,0.005,0.004,0.003,0.004,0.003,0.003,0.003,0.003,0.003,0.002,0.003,0.002,0.002,0.003,0.003,0.003,0.003,0.004,0.004,0.003,0.004,0.003,0.003,0.002,0.003,0.002,0.003,0.003,0.003,0.002,0.002,0.002,0.002,0.002,0.004
▁a,0.007,0.006,0.006,0.007,0.01,0.006,0.006,0.005,0.005,0.005,0.005,0.003,0.009,0.011,0.015,0.018,0.01,0.014,0.015,0.009,0.005,0.005,0.005,0.005,0.003,0.004,0.004,0.005,0.004,0.004,0.005,0.004,0.005,0.004,0.004,0.004,0.004,0.004,0.005,0.004,0.005,0.004,0.004,0.003,0.004,0.002,0.004,0.003,0.004,0.005,0.003,0.003,0.003,0.003,0.002,0.004,0.004
▁stand,0.008,0.01,0.007,0.009,0.011,0.009,0.01,0.01,0.008,0.009,0.006,0.007,0.014,0.021,0.046,0.079,0.04,0.04,0.023,0.012,0.007,0.009,0.01,0.008,0.006,0.006,0.006,0.006,0.005,0.006,0.005,0.004,0.005,0.004,0.004,0.004,0.004,0.005,0.007,0.005,0.006,0.005,0.007,0.005,0.004,0.003,0.005,0.004,0.005,0.005,0.005,0.004,0.004,0.004,0.003,0.004,0.005
-,0.003,0.005,0.003,0.004,0.004,0.003,0.003,0.003,0.003,0.003,0.003,0.002,0.005,0.006,0.016,0.074,0.027,0.013,0.007,0.004,0.004,0.003,0.003,0.002,0.002,0.002,0.003,0.002,0.003,0.003,0.003,0.002,0.006,0.004,0.003,0.002,0.003,0.002,0.002,0.002,0.002,0.002,0.003,0.002,0.002,0.005,0.004,0.002,0.002,0.002,0.002,0.001,0.001,0.002,0.002,0.002,0.002
up,0.01,0.013,0.011,0.011,0.018,0.014,0.012,0.012,0.01,0.012,0.009,0.009,0.02,0.026,0.043,0.091,0.074,0.072,0.034,0.018,0.01,0.011,0.016,0.011,0.008,0.007,0.008,0.009,0.006,0.008,0.006,0.006,0.006,0.005,0.005,0.006,0.006,0.007,0.008,0.007,0.01,0.007,0.009,0.007,0.006,0.005,0.006,0.005,0.006,0.006,0.006,0.006,0.004,0.005,0.004,0.005,0.006
▁comedy,0.02,0.024,0.02,0.017,0.029,0.019,0.02,0.021,0.019,0.024,0.015,0.02,0.026,0.027,0.053,0.027,0.019,0.093,0.049,0.026,0.013,0.019,0.02,0.014,0.01,0.014,0.016,0.014,0.008,0.012,0.013,0.015,0.011,0.008,0.009,0.009,0.011,0.01,0.014,0.013,0.014,0.013,0.01,0.008,0.01,0.007,0.011,0.007,0.011,0.009,0.012,0.007,0.006,0.008,0.006,0.006,0.011
▁transcript,0.028,0.031,0.022,0.033,0.038,0.025,0.025,0.024,0.018,0.021,0.013,0.011,0.039,0.055,0.134,0.034,0.031,0.092,0.13,0.044,0.027,0.025,0.02,0.014,0.012,0.012,0.013,0.013,0.009,0.012,0.008,0.009,0.013,0.011,0.01,0.012,0.012,0.015,0.017,0.015,0.015,0.009,0.013,0.012,0.014,0.007,0.011,0.007,0.013,0.01,0.009,0.008,0.009,0.008,0.007,0.009,0.016
:,0.016,0.013,0.012,0.014,0.011,0.011,0.008,0.008,0.01,0.008,0.006,0.006,0.01,0.019,0.015,0.011,0.009,0.01,0.013,0.014,0.016,0.01,0.009,0.008,0.005,0.006,0.008,0.008,0.005,0.007,0.008,0.006,0.01,0.008,0.006,0.008,0.007,0.01,0.008,0.008,0.007,0.004,0.006,0.005,0.007,0.004,0.006,0.004,0.006,0.006,0.005,0.003,0.005,0.005,0.004,0.006,0.009


In [31]:
out.sequence_attributions[0].target_attributions.sum(-1)

array([[ 9.3318298e-03, -2.5626723e-02, -2.4753304e-02, ...,
        -5.7015467e-02,  4.4034969e-06, -1.4212786e-04],
       [-8.6237743e-02,  5.4679070e-02, -2.1457385e-02, ...,
         6.9233894e-02, -2.9941932e-05, -2.6500195e-05],
       [ 1.3873137e-03,  2.1565673e-03, -4.9037402e-03, ...,
        -2.7292579e-02, -4.5716383e-06,  1.6739892e-05],
       ...,
       [           nan,            nan,            nan, ...,
                   nan, -1.1118839e-04,  2.4466758e-04],
       [           nan,            nan,            nan, ...,
                   nan,            nan, -3.3755862e-04],
       [           nan,            nan,            nan, ...,
                   nan,            nan,            nan]], dtype=float32)

In [10]:
for part in parts:
    prompt = f"This is a stand-up comedy transcript: '''{part}'''\nExtract the key humorous lines and punchlines:"

    out = model.attribute(
    prompt,
    n_steps=16,
    batch_size=1,
    device=model.device,
    generation_args=dict(
        max_new_tokens=65,
    )
    )

    out.show()

The model is loaded in 8bit mode. The device cannot be changed after loading the model.
Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 119/119 [00:28<00:00,  1.73it/s]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.


,,Sure,",",▁here,▁are,▁the,▁key,▁humorous,▁lines,▁and,▁punch,lines,▁from,▁the,▁transcript,:,,-,"▁""",It,▁was,▁like,▁the,▁worst,▁experience,▁of,▁my,▁life,".""",,-,"▁""",Don,',t,▁ever,▁do,▁that,▁with,▁your,▁mom,".""",,-,"▁""",It,▁was,▁so,▁emotional,"."""
<bos>,0.2,0.071,0.066,0.079,0.073,0.074,0.097,0.074,0.119,0.085,0.064,0.078,0.075,0.098,0.067,0.073,0.118,0.107,0.053,0.053,0.057,0.082,0.129,0.053,0.08,0.127,0.086,0.058,0.065,0.06,0.076,0.066,0.045,0.058,0.075,0.075,0.074,0.074,0.079,0.059,0.072,0.052,0.046,0.054,0.042,0.047,0.041,0.054,0.064,0.055
This,0.111,0.042,0.025,0.019,0.022,0.026,0.011,0.013,0.019,0.01,0.037,0.037,0.024,0.019,0.02,0.034,0.065,0.017,0.011,0.023,0.018,0.006,0.044,0.01,0.025,0.037,0.012,0.025,0.01,0.018,0.052,0.036,0.008,0.011,0.025,0.011,0.033,0.027,0.019,0.011,0.016,0.009,0.013,0.018,0.024,0.007,0.013,0.006,0.016,0.031
▁is,0.008,0.006,0.006,0.008,0.007,0.006,0.006,0.005,0.005,0.005,0.003,0.003,0.008,0.012,0.013,0.007,0.005,0.006,0.007,0.011,0.006,0.004,0.005,0.005,0.003,0.004,0.004,0.004,0.004,0.004,0.004,0.003,0.004,0.005,0.004,0.003,0.004,0.003,0.003,0.004,0.003,0.004,0.004,0.003,0.004,0.003,0.005,0.003,0.003,0.003
▁a,0.009,0.007,0.007,0.009,0.01,0.007,0.007,0.006,0.006,0.006,0.005,0.003,0.01,0.012,0.016,0.008,0.006,0.006,0.008,0.012,0.006,0.005,0.006,0.008,0.004,0.005,0.005,0.004,0.005,0.005,0.004,0.004,0.005,0.007,0.007,0.005,0.004,0.004,0.005,0.004,0.004,0.004,0.005,0.004,0.005,0.005,0.004,0.005,0.004,0.004
▁stand,0.01,0.01,0.008,0.013,0.012,0.011,0.012,0.012,0.009,0.01,0.006,0.006,0.016,0.023,0.046,0.01,0.006,0.008,0.015,0.026,0.011,0.006,0.007,0.009,0.007,0.007,0.009,0.006,0.007,0.007,0.004,0.006,0.01,0.011,0.011,0.007,0.009,0.008,0.005,0.01,0.007,0.006,0.008,0.006,0.008,0.009,0.006,0.005,0.008,0.006
-,0.004,0.004,0.003,0.004,0.004,0.004,0.004,0.003,0.003,0.003,0.003,0.003,0.005,0.008,0.014,0.004,0.003,0.004,0.005,0.007,0.004,0.002,0.003,0.003,0.003,0.002,0.003,0.002,0.003,0.003,0.004,0.003,0.003,0.011,0.004,0.003,0.003,0.003,0.002,0.003,0.003,0.002,0.003,0.004,0.002,0.003,0.003,0.002,0.003,0.003
up,0.013,0.017,0.01,0.017,0.018,0.017,0.016,0.014,0.01,0.012,0.009,0.009,0.024,0.03,0.043,0.018,0.008,0.011,0.023,0.038,0.015,0.009,0.008,0.012,0.008,0.01,0.011,0.009,0.01,0.01,0.006,0.01,0.015,0.015,0.019,0.009,0.013,0.012,0.007,0.013,0.012,0.011,0.012,0.01,0.011,0.013,0.009,0.008,0.011,0.008
▁comedy,0.021,0.027,0.02,0.023,0.031,0.025,0.025,0.025,0.019,0.024,0.016,0.016,0.028,0.033,0.054,0.028,0.014,0.019,0.023,0.036,0.015,0.011,0.014,0.012,0.012,0.013,0.016,0.008,0.015,0.015,0.011,0.011,0.012,0.02,0.014,0.01,0.013,0.011,0.011,0.017,0.016,0.013,0.021,0.016,0.017,0.013,0.01,0.009,0.014,0.011
▁transcript,0.035,0.029,0.021,0.037,0.045,0.03,0.029,0.029,0.021,0.024,0.015,0.013,0.043,0.06,0.156,0.047,0.03,0.024,0.042,0.071,0.023,0.016,0.018,0.014,0.015,0.016,0.016,0.011,0.021,0.02,0.015,0.014,0.015,0.027,0.015,0.015,0.014,0.018,0.015,0.015,0.012,0.019,0.025,0.02,0.021,0.015,0.014,0.017,0.014,0.015
:,0.019,0.012,0.016,0.015,0.012,0.011,0.01,0.011,0.012,0.01,0.009,0.009,0.011,0.015,0.012,0.015,0.019,0.01,0.015,0.019,0.009,0.007,0.01,0.008,0.007,0.008,0.007,0.006,0.009,0.009,0.007,0.009,0.006,0.015,0.006,0.006,0.007,0.009,0.007,0.009,0.006,0.01,0.01,0.009,0.009,0.006,0.008,0.006,0.005,0.007


The model is loaded in 8bit mode. The device cannot be changed after loading the model.
Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 142/142 [00:31<00:00,  1.83it/s]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.


,,Sure,",",▁here,▁are,▁the,▁key,▁humorous,▁lines,▁and,▁punch,lines,▁from,▁the,▁stand,-,up,▁comedy,▁transcript,:,,-,▁She,▁refused,▁to,▁let,▁go,▁of,▁a,▁Texas,▁Instruments,▁TI,-,8,2,…,▁manual,.,,-,▁Those,▁of,▁you,▁under,▁,2,5,▁probably,▁don,’,t,▁know,▁what,▁that,▁calculator,▁is,.
<bos>,0.2,0.067,0.077,0.103,0.063,0.072,0.07,0.068,0.114,0.093,0.052,0.074,0.082,0.088,0.05,0.078,0.085,0.067,0.081,0.071,0.13,0.095,0.054,0.068,0.064,0.083,0.055,0.057,0.088,0.071,0.076,0.046,0.068,0.062,0.076,0.038,0.07,0.076,0.039,0.04,0.041,0.062,0.059,0.062,0.071,0.086,0.055,0.048,0.052,0.038,0.057,0.08,0.062,0.101,0.042,0.072,0.048
This,0.113,0.028,0.029,0.038,0.023,0.03,0.013,0.012,0.019,0.009,0.035,0.036,0.026,0.028,0.023,0.024,0.061,0.017,0.033,0.021,0.072,0.021,0.009,0.006,0.018,0.006,0.028,0.022,0.006,0.007,0.027,0.023,0.046,0.04,0.021,0.005,0.008,0.036,0.014,0.03,0.01,0.012,0.018,0.029,0.052,0.006,0.032,0.025,0.034,0.01,0.039,0.006,0.021,0.008,0.019,0.026,0.031
▁is,0.006,0.006,0.006,0.006,0.007,0.006,0.005,0.005,0.004,0.004,0.003,0.002,0.008,0.012,0.012,0.012,0.008,0.009,0.015,0.008,0.005,0.004,0.005,0.004,0.003,0.004,0.003,0.003,0.003,0.003,0.003,0.002,0.003,0.002,0.002,0.003,0.003,0.003,0.003,0.004,0.004,0.003,0.004,0.003,0.003,0.002,0.003,0.002,0.003,0.003,0.003,0.002,0.002,0.002,0.002,0.002,0.004
▁a,0.007,0.006,0.006,0.007,0.01,0.006,0.006,0.005,0.005,0.005,0.005,0.003,0.009,0.011,0.015,0.018,0.01,0.014,0.015,0.009,0.005,0.005,0.005,0.005,0.003,0.004,0.004,0.005,0.004,0.004,0.005,0.004,0.005,0.004,0.004,0.004,0.004,0.004,0.005,0.004,0.005,0.004,0.004,0.003,0.004,0.002,0.004,0.003,0.004,0.005,0.003,0.003,0.003,0.003,0.002,0.004,0.004
▁stand,0.008,0.01,0.007,0.009,0.011,0.009,0.01,0.01,0.008,0.009,0.006,0.007,0.014,0.021,0.046,0.079,0.04,0.04,0.023,0.012,0.007,0.009,0.01,0.008,0.006,0.006,0.006,0.006,0.005,0.006,0.005,0.004,0.005,0.004,0.004,0.004,0.004,0.005,0.007,0.005,0.006,0.005,0.007,0.005,0.004,0.003,0.005,0.004,0.005,0.005,0.005,0.004,0.004,0.004,0.003,0.004,0.005
-,0.003,0.005,0.003,0.004,0.004,0.003,0.003,0.003,0.003,0.003,0.003,0.002,0.005,0.006,0.016,0.074,0.027,0.013,0.007,0.004,0.004,0.003,0.003,0.002,0.002,0.002,0.003,0.002,0.003,0.003,0.003,0.002,0.006,0.004,0.003,0.002,0.003,0.002,0.002,0.002,0.002,0.002,0.003,0.002,0.002,0.005,0.004,0.002,0.002,0.002,0.002,0.001,0.001,0.002,0.002,0.002,0.002
up,0.01,0.013,0.011,0.011,0.018,0.014,0.012,0.012,0.01,0.012,0.009,0.009,0.02,0.026,0.043,0.091,0.074,0.072,0.034,0.018,0.01,0.011,0.016,0.011,0.008,0.007,0.008,0.009,0.006,0.008,0.006,0.006,0.006,0.005,0.005,0.006,0.006,0.007,0.008,0.007,0.01,0.007,0.009,0.007,0.006,0.005,0.006,0.005,0.006,0.006,0.006,0.006,0.004,0.005,0.004,0.005,0.006
▁comedy,0.02,0.024,0.02,0.017,0.029,0.019,0.02,0.021,0.019,0.024,0.015,0.02,0.026,0.027,0.053,0.027,0.019,0.093,0.049,0.026,0.013,0.019,0.02,0.014,0.01,0.014,0.016,0.014,0.008,0.012,0.013,0.015,0.011,0.008,0.009,0.009,0.011,0.01,0.014,0.013,0.014,0.013,0.01,0.008,0.01,0.007,0.011,0.007,0.011,0.009,0.012,0.007,0.006,0.008,0.006,0.006,0.011
▁transcript,0.028,0.031,0.022,0.033,0.038,0.025,0.025,0.024,0.018,0.021,0.013,0.011,0.039,0.055,0.134,0.034,0.031,0.092,0.13,0.044,0.027,0.025,0.02,0.014,0.012,0.012,0.013,0.013,0.009,0.012,0.008,0.009,0.013,0.011,0.01,0.012,0.012,0.015,0.017,0.015,0.015,0.009,0.013,0.012,0.014,0.007,0.011,0.007,0.013,0.01,0.009,0.008,0.009,0.008,0.007,0.009,0.016
:,0.016,0.013,0.012,0.014,0.011,0.011,0.008,0.008,0.01,0.008,0.006,0.006,0.01,0.019,0.015,0.011,0.009,0.01,0.013,0.014,0.016,0.01,0.009,0.008,0.005,0.006,0.008,0.008,0.005,0.007,0.008,0.006,0.01,0.008,0.006,0.008,0.007,0.01,0.008,0.008,0.007,0.004,0.006,0.005,0.007,0.004,0.006,0.004,0.006,0.006,0.005,0.003,0.005,0.005,0.004,0.006,0.009


The model is loaded in 8bit mode. The device cannot be changed after loading the model.
Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 119/119 [00:26<00:00,  1.74it/s]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.


,,Sure,",",▁here,▁are,▁the,▁key,▁humorous,▁lines,▁and,▁punch,lines,▁from,▁the,▁transcript,:,,-,▁It,▁was,▁this,▁calculator,▁that,▁bamboo,z,led,▁my,▁generation,.,,-,▁Everybody,▁thought,▁it,▁was,▁like,▁this,▁Judy,▁Jet,son,’,s,▁laptop,▁from,▁the,▁future,.
<bos>,0.2,0.076,0.068,0.089,0.065,0.07,0.086,0.066,0.116,0.08,0.053,0.062,0.089,0.078,0.069,0.063,0.146,0.098,0.056,0.073,0.042,0.048,0.078,0.069,0.04,0.039,0.063,0.069,0.079,0.045,0.038,0.04,0.176,0.095,0.08,0.059,0.061,0.062,0.063,0.04,0.04,0.044,0.047,0.063,0.078,0.062,0.069
This,0.099,0.027,0.028,0.037,0.027,0.033,0.01,0.012,0.02,0.011,0.035,0.034,0.025,0.022,0.02,0.033,0.066,0.014,0.01,0.015,0.021,0.013,0.031,0.016,0.013,0.011,0.022,0.024,0.022,0.018,0.023,0.007,0.006,0.019,0.03,0.011,0.012,0.029,0.029,0.009,0.014,0.022,0.03,0.023,0.031,0.037,0.051
▁is,0.006,0.007,0.007,0.007,0.008,0.007,0.006,0.005,0.004,0.004,0.003,0.002,0.008,0.012,0.012,0.007,0.005,0.006,0.006,0.007,0.011,0.006,0.006,0.004,0.004,0.004,0.004,0.004,0.004,0.004,0.005,0.004,0.003,0.003,0.003,0.003,0.003,0.003,0.002,0.002,0.003,0.003,0.002,0.002,0.003,0.002,0.003
▁a,0.008,0.007,0.007,0.008,0.01,0.007,0.007,0.005,0.005,0.006,0.005,0.003,0.01,0.013,0.015,0.007,0.006,0.007,0.006,0.007,0.009,0.007,0.007,0.006,0.005,0.005,0.005,0.004,0.004,0.006,0.005,0.005,0.004,0.004,0.003,0.005,0.003,0.004,0.004,0.003,0.004,0.003,0.003,0.003,0.003,0.003,0.004
▁stand,0.009,0.012,0.008,0.012,0.011,0.011,0.012,0.012,0.009,0.01,0.007,0.006,0.017,0.025,0.042,0.01,0.008,0.011,0.013,0.013,0.009,0.01,0.008,0.006,0.005,0.008,0.007,0.009,0.007,0.008,0.008,0.006,0.005,0.007,0.005,0.006,0.005,0.005,0.005,0.003,0.008,0.005,0.005,0.004,0.004,0.004,0.006
-,0.004,0.005,0.003,0.004,0.004,0.004,0.004,0.003,0.003,0.004,0.003,0.003,0.005,0.008,0.015,0.003,0.003,0.004,0.004,0.004,0.003,0.003,0.003,0.004,0.005,0.006,0.003,0.004,0.002,0.003,0.003,0.002,0.003,0.002,0.002,0.002,0.002,0.002,0.002,0.004,0.002,0.002,0.002,0.002,0.002,0.002,0.002
up,0.011,0.016,0.012,0.015,0.017,0.015,0.017,0.014,0.012,0.012,0.011,0.008,0.024,0.033,0.043,0.017,0.01,0.014,0.017,0.018,0.012,0.014,0.012,0.009,0.006,0.011,0.011,0.011,0.01,0.012,0.009,0.009,0.007,0.011,0.006,0.009,0.006,0.006,0.007,0.004,0.008,0.007,0.008,0.005,0.005,0.006,0.008
▁comedy,0.023,0.027,0.02,0.021,0.025,0.02,0.021,0.023,0.018,0.024,0.015,0.023,0.028,0.032,0.053,0.027,0.013,0.022,0.02,0.017,0.015,0.018,0.015,0.016,0.014,0.014,0.013,0.015,0.014,0.019,0.014,0.013,0.009,0.009,0.011,0.011,0.009,0.011,0.009,0.008,0.013,0.008,0.008,0.007,0.01,0.01,0.009
▁transcript,0.034,0.033,0.021,0.037,0.038,0.03,0.028,0.025,0.023,0.025,0.014,0.01,0.047,0.064,0.15,0.047,0.027,0.033,0.03,0.021,0.017,0.022,0.02,0.014,0.012,0.011,0.014,0.015,0.019,0.021,0.021,0.018,0.012,0.014,0.013,0.01,0.011,0.012,0.012,0.008,0.016,0.01,0.011,0.011,0.01,0.011,0.013
:,0.017,0.014,0.013,0.013,0.01,0.009,0.01,0.009,0.01,0.009,0.006,0.006,0.011,0.018,0.012,0.015,0.019,0.011,0.011,0.011,0.009,0.009,0.009,0.006,0.008,0.006,0.008,0.008,0.008,0.01,0.009,0.006,0.004,0.005,0.006,0.005,0.005,0.005,0.005,0.004,0.008,0.004,0.005,0.005,0.005,0.005,0.007


The model is loaded in 8bit mode. The device cannot be changed after loading the model.
Unused arguments during attribution: {'n_steps': 16}
Attributing with input_x_gradient...: 100%|██████████| 194/194 [00:41<00:00,  1.57it/s]
The model is loaded in 8bit mode. The device cannot be changed after loading the model.


,,Sure,",",▁here,▁are,▁the,▁key,▁humorous,▁lines,▁and,▁punch,lines,▁from,▁the,▁stand,-,up,▁comedy,▁transcript,:,,-,"▁""",It,▁was,▁like,▁the,▁Tesla,▁of,▁my,▁time,".""",,-,"▁""",My,▁mom,▁got,▁so,▁emotional,▁about,▁the,▁manual,▁and,▁she,▁was,▁like,",",▁‘,You,▁never,▁know,▁when,▁you,▁might,▁need,▁this,.’”,,-,"▁""",I,’,m,▁not
<bos>,0.198,0.064,0.066,0.082,0.083,0.076,0.082,0.055,0.098,0.058,0.057,0.066,0.088,0.086,0.053,0.077,0.079,0.061,0.082,0.094,0.131,0.103,0.045,0.079,0.047,0.063,0.102,0.087,0.091,0.07,0.079,0.041,0.037,0.05,0.073,0.051,0.052,0.091,0.115,0.103,0.139,0.071,0.058,0.112,0.078,0.072,0.131,0.081,0.063,0.061,0.077,0.084,0.046,0.074,0.037,0.057,0.041,0.033,0.038,0.036,0.022,0.026,0.025,0.035,0.031
This,0.108,0.037,0.022,0.037,0.019,0.025,0.013,0.015,0.016,0.008,0.033,0.038,0.024,0.028,0.024,0.022,0.065,0.018,0.032,0.016,0.061,0.017,0.012,0.009,0.013,0.016,0.013,0.034,0.01,0.016,0.025,0.013,0.018,0.021,0.047,0.009,0.006,0.006,0.02,0.009,0.01,0.023,0.011,0.007,0.011,0.005,0.016,0.016,0.016,0.011,0.005,0.012,0.011,0.01,0.008,0.007,0.009,0.006,0.026,0.006,0.004,0.006,0.008,0.005,0.004
▁is,0.005,0.006,0.007,0.007,0.007,0.006,0.005,0.005,0.006,0.004,0.003,0.002,0.007,0.011,0.012,0.012,0.007,0.01,0.014,0.007,0.005,0.005,0.004,0.006,0.01,0.013,0.004,0.004,0.005,0.005,0.004,0.006,0.004,0.003,0.003,0.004,0.003,0.003,0.004,0.003,0.003,0.003,0.003,0.004,0.003,0.003,0.002,0.002,0.002,0.003,0.003,0.002,0.002,0.002,0.003,0.002,0.003,0.002,0.001,0.002,0.001,0.003,0.003,0.002,0.002
▁a,0.007,0.006,0.006,0.008,0.009,0.006,0.006,0.005,0.006,0.005,0.005,0.003,0.009,0.012,0.016,0.018,0.01,0.014,0.014,0.008,0.005,0.005,0.005,0.006,0.008,0.011,0.005,0.006,0.005,0.005,0.005,0.007,0.005,0.004,0.003,0.004,0.004,0.005,0.004,0.005,0.004,0.005,0.006,0.004,0.003,0.004,0.003,0.003,0.003,0.003,0.003,0.002,0.003,0.002,0.003,0.003,0.003,0.002,0.002,0.003,0.002,0.004,0.004,0.003,0.002
▁stand,0.008,0.011,0.008,0.009,0.011,0.01,0.009,0.011,0.008,0.01,0.006,0.006,0.014,0.022,0.046,0.076,0.043,0.043,0.025,0.011,0.005,0.009,0.01,0.01,0.011,0.007,0.008,0.006,0.006,0.006,0.007,0.01,0.005,0.006,0.005,0.006,0.007,0.007,0.007,0.007,0.005,0.005,0.006,0.004,0.004,0.006,0.004,0.005,0.004,0.004,0.005,0.004,0.003,0.004,0.003,0.003,0.004,0.004,0.002,0.004,0.002,0.007,0.006,0.006,0.003
-,0.003,0.004,0.003,0.003,0.003,0.003,0.004,0.003,0.003,0.004,0.002,0.003,0.005,0.007,0.015,0.076,0.026,0.014,0.007,0.004,0.004,0.003,0.003,0.003,0.003,0.002,0.003,0.003,0.003,0.002,0.002,0.003,0.002,0.004,0.002,0.002,0.002,0.002,0.003,0.003,0.002,0.003,0.004,0.002,0.002,0.002,0.001,0.002,0.002,0.002,0.002,0.001,0.002,0.002,0.002,0.002,0.002,0.001,0.001,0.004,0.001,0.002,0.002,0.002,0.001
up,0.01,0.013,0.013,0.011,0.015,0.015,0.013,0.012,0.01,0.011,0.009,0.008,0.02,0.029,0.044,0.098,0.076,0.078,0.035,0.018,0.007,0.012,0.013,0.014,0.017,0.01,0.011,0.009,0.01,0.008,0.008,0.013,0.008,0.008,0.008,0.008,0.01,0.009,0.008,0.01,0.008,0.008,0.008,0.005,0.006,0.008,0.005,0.006,0.006,0.006,0.007,0.005,0.004,0.005,0.004,0.006,0.006,0.006,0.004,0.007,0.004,0.008,0.008,0.007,0.005
▁comedy,0.018,0.023,0.019,0.015,0.024,0.018,0.017,0.019,0.018,0.021,0.015,0.015,0.023,0.031,0.048,0.018,0.019,0.088,0.05,0.023,0.012,0.019,0.012,0.016,0.017,0.015,0.018,0.014,0.013,0.012,0.012,0.015,0.013,0.014,0.01,0.012,0.014,0.009,0.01,0.014,0.012,0.014,0.017,0.008,0.009,0.007,0.006,0.007,0.006,0.008,0.01,0.006,0.008,0.007,0.006,0.011,0.012,0.006,0.005,0.008,0.006,0.007,0.01,0.008,0.006
▁transcript,0.025,0.027,0.019,0.026,0.037,0.021,0.022,0.023,0.021,0.021,0.013,0.011,0.041,0.056,0.127,0.035,0.026,0.083,0.126,0.036,0.024,0.023,0.021,0.017,0.023,0.016,0.014,0.014,0.01,0.016,0.012,0.021,0.014,0.014,0.014,0.013,0.009,0.014,0.009,0.01,0.01,0.012,0.013,0.008,0.01,0.011,0.01,0.011,0.01,0.012,0.008,0.006,0.008,0.007,0.006,0.008,0.01,0.007,0.006,0.011,0.006,0.014,0.013,0.01,0.007
:,0.017,0.016,0.011,0.013,0.012,0.011,0.008,0.01,0.012,0.01,0.007,0.007,0.01,0.017,0.015,0.009,0.008,0.01,0.013,0.01,0.014,0.009,0.008,0.008,0.011,0.00